# AI Agents for Network Engineering

This (Jupyter Notebook) is an interactive coding environment that lets you write and run Python code in a web-based interface. Simply click on the play button or [SHIFT]+[ENTER] to execute the cell.

In [ ]:
print("hello")

## 1. Test the Local Large Language Model with Ollama

Let's see how the local large language model is working and the duration time of the prompt execution.

## 2. Create Your First AI Agent with `smolagents`

Let's start writing our first AI agent in this framework and learn more about its components.

## 3. Important Building Blocks for AI Agents

In order to understand AI agents, it is important to know these building blocks of AI agentic frameworks:

* Understanding the Agentic Behavior using OpenTelemetry
* Function/Tool Calling
* Structured Output

### 3.1 Using OpenTelemetry for Understanding & Debugging

After starting the OpenTelemetry server, you can access it at [http://0.0.0.0:6006/projects/](http://0.0.0.0:6006/projects/) and run this code below. The next time you are executing a smolagent, you will see data in the dashboard.

### 3.2. Function/Tool Calling with LLMs

Predefined functions:

In [ ]:
from smolagents import tool

@tool
def get_username_password_for_device(ip_address:str) -> str:
    """
    Returns the username and password separated by a comma for the given ip address or hostname.

    Args:
        ip_address: The IP address or hostname of the device

    Returns:
        str: A string with the username and password separated by a comma.
    """
    import json
    
    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        return f"{devices[ip_address]['username']},{devices[ip_address]['password']}"

@tool
def show_ip_route(host:str,username:str,password:str,device_type:str = "cisco_ios",) -> str:
    """
    Return the routing table of the device. Executes the command 'show ip route'.

    Args:
        host: The IP address or hostname of the device
        username: The username for the device
        password: The password for the device
        device_type: The device type for the device

    Returns:
        str: The routing table of the device.
    """
    from netmiko import ConnectHandler    
                                                                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': device_type                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)                                                                                                                                                                                                                                                                                     
                                                                                                                                                                                
    # Execute command to retrieve running configuration                                                                                                                        
    running_config_output = connection.send_command('show ip route')                                                                                                     
                                                                                                                                                                                
    return running_config_output

Now, let's add these functions to the CodeAgent and authorize to import these Python libraries in the sandboxed Python environment of the agent. Let's run this example with the following prompt.

### 3.3 Structured Output with LLMs

In [ ]:
def run_ios_show_command_on_device_trusted(show_command:str,host:str) -> str:
    """
    Returns the output of the provided show command from the device with the given host, username, password, and device type.

    Args:
        show_command: The show command to run on the device
        host: The IP address or hostname of the device
        device_type: The device type for the device

    Returns:
        str: Output of the provided show command
    """
    print("Running show command on device...")

    from netmiko import ConnectHandler
    import json

    with open("hosts.json", 'r') as file:
        devices = json.load(file)
        username = devices[host]["username"]
        password = devices[host]["password"]
                                                          
    device = {                                                                                                                                                                 
        'ip': host,                                                                                                                                                   
        'username': username,                                                                                                                                                  
        'password': password,                                                                                                                                                  
        'device_type': "cisco_ios",                                                                                        
    }                                                                                                                                                                          
                                                                                                                                                                                
    # Establish connection to the device                                                                                                                                       
    connection = ConnectHandler(**device)

    if show_command.startswith("show") or show_command.startswith("sh"):
        # Execute command to retrieve running configuration                                                                                                                        
        running_config_output = connection.send_command(show_command)
        print(running_config_output)
        return running_config_output
    else:
        return "Error! You are only allowed to run show commands. Try again and use a show command."

Our pydantic output model:

Then let's define our prompts: system + user prompt and tell the LLM how to structure the output and run the query:

In [ ]:
from ollama import chat

response = chat(
    messages=[
        {
        'role': 'system',
        'content': 'You are a helpful networking assistant.',
    },
    {
        'role': 'user',
        'content': f'''What can you say about my Cisco switch?
                    Here is the output of the "show version" command:
                    {run_ios_show_command_on_device_trusted(ios_show_command,device_host)}''',
    }
    ],
    model='llama3.1',
    format=Cat8000.model_json_schema(),
)

cat8000_instance = Cat8000.model_validate_json(response.message.content)

Output:

# Next steps

Check out all the other use-cases! Run them and go through the code to understand the concept and how you can apply them to your environment and use-cases.